虽然 pandas 采用了 NumPy 的许多编码习语，但最大的区别是 pandas 是为处理表格或异构数据(tabular or heterogeneous data)而设计的。相比之下，NumPy 最适合处理同质(homogeneous)类型的数值数组数据。

In [1]:
import numpy as np 
import pandas as pd 
from pandas import Series, DataFrame

# 5.1 Introduction to pandas Data Structures

## Series

In [2]:
obj = pd.Series([4, 7, -5, 3])  
obj

0    4
1    7
2   -5
3    3
dtype: int64

In [4]:
obj.array

<NumpyExtensionArray>
[np.int64(4), np.int64(7), np.int64(-5), np.int64(3)]
Length: 4, dtype: int64

In [5]:
obj.index

RangeIndex(start=0, stop=4, step=1)

In [6]:
obj2 = pd.Series([3, 4, 2, -7], index=['d', 'b', 'c', 'a'])
obj2


d    3
b    4
c    2
a   -7
dtype: int64

In [7]:
obj2.index

Index(['d', 'b', 'c', 'a'], dtype='object')

In [ ]:
print(obj2['a'])
print(obj2[['d', 'b']]) # 输入一个list

-7
d    3
b    4
dtype: int64


使用布尔数组过滤、标量乘法或应用数学函数，将保留索引值链接：

In [12]:
obj2[obj2 > 0]

d    3
b    4
c    2
dtype: int64

In [13]:
obj2*2

d     6
b     8
c     4
a   -14
dtype: int64

In [14]:
np.exp(obj2)

d    20.085537
b    54.598150
c     7.389056
a     0.000912
dtype: float64

In [16]:
print('b' in obj2)
print('e' in obj2)

True
False


In [23]:
# 如果有现成的dict 数据，可以直接传递 dict 创建 Series
sdata = {"Ohio": 35000, "Texas": 71000, "Oregon": 16000, "Utah": 5000}
obj3 = pd.Series(sdata)
obj3


Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64

In [24]:
# 从 Series convert to dict
obj3.to_dict(sdata)

/var/folders/1m/2jsf455s4r7dx9t8l233n55c0000gp/T/ipykernel_24017/2867738030.py:2: FutureWarning: Starting with pandas version 3.0 all arguments of to_dict will be keyword-only.
  obj3.to_dict(sdata)


{'Ohio': 35000, 'Texas': 71000, 'Oregon': 16000, 'Utah': 5000}

当您只传递字典时，生成的 Series 中的索引将根据字典的 keys 方法（取决于键的插入顺序）遵循键的顺序。您可以通过按照您希望它们出现在生成的 Series 中的顺序传递带有字典键的索引来覆盖此设置：

In [25]:
states = ["California", "Ohio", "Oregon", "Texas"]
obj4 = pd.Series(sdata, index=states)
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

在 sdata 中找到的三个值被放置在适当的位置，但由于没有找到 “California” 的值，它显示为 NaN（不是数字），这在 pandas 中被认为是标记缺失值或 NA 值。由于 “犹他州” 未包含在各州中，因此将其排除在生成的对象之外。

In [26]:
print(pd.isna(obj4)) # 检查是否为缺失值
print(pd.notna(obj4)) # 检查是否不为缺失值

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool
California    False
Ohio           True
Oregon         True
Texas          True
dtype: bool


In [27]:
obj4.isna() # 也可以用实例方法的方式使用

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

In [ ]:
print([obj3, obj4])
obj3 + obj4 # 后续讨论数据对齐逻辑

[Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64, California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64]


California         NaN
Ohio           70000.0
Oregon         32000.0
Texas         142000.0
Utah               NaN
dtype: float64

In [ ]:
# 为 Series 增加名称Attribute
obj4.name = "state_population"
obj4.index.name = "state"
obj4

state
California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
Name: state_population, dtype: float64

In [ ]:
print(obj)
obj.index = ["CA", "OH", "OR", "TX"] # 重新赋值index
obj


0    4
1    7
2   -5
3    3
dtype: int64


CA    4
OH    7
OR   -5
TX    3
dtype: int64

## Dataframe